# Index Paladin Database

Jacobo de la Cuesta-Zuluaga. June 2025.

The aim of this notebook is to index the `UHGG` protein catalog for use with `Paladin`

## Load libraries and set paths

In [1]:
# Libraries
library(tidyverse)
library(conflicted)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# Solve conflicts
conflicts_prefer(dplyr::filter)

[conflicted] Will prefer dplyr::filter over any other package.


In [3]:
# Directories
# Base directory
databases_dir = "/mnt/lustre/groups/maier/databases"

# UHGG database
uhgg_faa = file.path(databases_dir, "UHGG/Protein_catalog/uhgp-90/uhgp-90.faa")

# Out
index_dir = file.path(databases_dir, "Paladin")
dir.create(index_dir)

# Conda
conda_env = "paladin"

Warning message in dir.create(index_dir):
“'/mnt/lustre/groups/maier/databases/Paladin' already exists”


## Index protein database

In [4]:
# Copy UHGG  fasta to paladin folder
file.copy(from = uhgg_faa, to = index_dir, overwrite = FALSE)
uhgg_ref_faa = file.path(index_dir, "uhgp-90.faa")

[1] FALSE

In [5]:
index_slurm_raw = str_glue(.open = "[", .close = "]",
"#!/bin/bash
##############################
#       Parameters           #
##############################

# This section will tell the cluster what are the resources your job will need.
# Change the parameters accordingly and carefully!
# The parameters here are a sensible start.

# Name of the job
#SBATCH --job-name=[[job_name]]

# Generate an output file and give it a name
#SBATCH --output=%x-%j.out

# Number of tasks
#SBATCH --ntasks=1

# Number of cpus that this task will need
#SBATCH --cpus-per-task=32

# Specify the total memory required per node
#SBATCH --mem=256G

# Specify the maximum time this job can take to run before being killed (hh:mm:ss)
#SBATCH --time=23:59:59

# job information
scontrol show job ${SLURM_JOB_ID}

# do your real computation
source $HOME/.bashrc
conda activate [[conda_env]]
cd [[index_dir]]
paladin index -r3 [[Protein_reference]]
")

In [6]:
index_slurm = str_glue(index_slurm_raw,
        job_name = "paladin_index", 
        index_dir = index_dir,
        Protein_reference = uhgg_ref_faa,
        conda_env = conda_env,
        .open = "[", .close = "]") 

index_slurm %>%
        print()

#!/bin/bash
##############################
#       Parameters           #
##############################

# This section will tell the cluster what are the resources your job will need.
# Change the parameters accordingly and carefully!
# The parameters here are a sensible start.

# Name of the job
#SBATCH --job-name=paladin_index

# Generate an output file and give it a name
#SBATCH --output=%x-%j.out

# Number of tasks
#SBATCH --ntasks=1

# Number of cpus that this task will need
#SBATCH --cpus-per-task=32

# Specify the total memory required per node
#SBATCH --mem=256G

# Specify the maximum time this job can take to run before being killed (hh:mm:ss)
#SBATCH --time=23:59:59

# job information
scontrol show job ${SLURM_JOB_ID}

# do your real computation
source $HOME/.bashrc
conda activate paladin
cd /mnt/lustre/groups/maier/databases/Paladin
paladin index -r3 /mnt/lustre/groups/maier/databases/Paladin/uhgp-90.faa


In [8]:
# Write file
index_slurmfile = file.path(index_dir, "index_slurm.sh")
write_lines(index_slurm, index_slurmfile)